<a href="https://colab.research.google.com/github/ranjan717/Beyond-Numbers/blob/main/Tableau_MetaData_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import xml.etree.ElementTree as ET

In [15]:
tableau_server = "https://prod-apnortheast-a.online.tableau.com"
username = "jabocih360@luravel.com"
password = "*vbFNrM5RAiSCbM"
site_id = "jabocih360ecd23eb245"


In [16]:
login_url = f"{tableau_server}/api/3.10/auth/signin"

In [17]:
payload = {
    "credentials": {
        "name": username,
        "password": password,
        "site": {
            "contentUrl": site_id
        }
    }
}

In [18]:
response = requests.post(login_url, json=payload)

In [19]:
if response.status_code == 200:
    try:

        root = ET.fromstring(response.content)
        token = root.find(".//{http://tableau.com/api}credentials").attrib['token']
        site_id = root.find(".//{http://tableau.com/api}site").attrib['id']
        print(f"Auth Token: {token}")
        print(f"Site ID: {site_id}")
    except ET.ParseError as e:
        print("Error parsing XML response:", e)
        print("Response content:", response.content)
        exit(1)
else:
    print("Failed to authenticate. Status code:", response.status_code)
    print("Response content:", response.content)
    exit(1)

Auth Token: mdvzxeblS3iZTXDarAC_zQ|FmVCqTUoNg96QgCoHXSQa8OcbJ1HyvBs|8f780486-5c83-45aa-afca-3f317f2accdb
Site ID: 8f780486-5c83-45aa-afca-3f317f2accdb


In [20]:
workbooks_url = f"{tableau_server}/api/3.10/sites/{site_id}/workbooks"

In [21]:
headers = {
    "X-Tableau-Auth": token
}

In [22]:
response = requests.get(workbooks_url, headers=headers)


In [23]:
if response.status_code == 200:
    try:
        root = ET.fromstring(response.content)
        workbooks = root.findall(".//{http://tableau.com/api}workbook")
        print("Workbooks:")
        workbook_id = None
        for workbook in workbooks:
            name = workbook.attrib['name']
            workbook_id_tmp = workbook.attrib['id']
            print(f"Name: {name}, ID: {workbook_id_tmp}")
            if name == 'Account Engagement':
                workbook_id = workbook_id_tmp
        if workbook_id:
            print(f"Found workbook 'Account Engagement' with ID: {workbook_id}")
        else:
            print("Workbook 'Account Engagement' not found.")
            exit(1)
    except ET.ParseError as e:
        print("Error parsing XML response:", e)
        print("Response content:", response.content)
else:
    print("Failed to list workbooks. Status code:", response.status_code)
    print("Response content:", response.content)
    exit(1)

Workbooks:
Name: Superstore, ID: e72c4cd7-e627-47d4-af4b-af744a381376
Name: World Indicators, ID: f552be47-668f-4051-bd1e-17904e712eb4
Name: Account Engagement, ID: e3f507fd-12b9-4ab4-8d59-a8c1a613d58f
Name: RETAIL%20-%20Retail%20Sales, ID: ea2298c8-aa83-40e1-8835-095899a8e1fa
Name: Ecommerce Site Sales analysis, ID: cad1de55-c1ba-4415-ba2c-4e1ec1046541
Found workbook 'Account Engagement' with ID: e3f507fd-12b9-4ab4-8d59-a8c1a613d58f


In [24]:


if workbook_id:
    metadata_url = f"{tableau_server}/api/3.10/sites/{site_id}/workbooks/{workbook_id}"

    response = requests.get(metadata_url, headers=headers)

    if response.status_code == 200:
        try:
            print("Response content:", response.content)

            root = ET.fromstring(response.content)
            workbook_metadata = {}

            workbook = root.find(".//{http://tableau.com/api}workbook")
            if workbook is not None:
                workbook_metadata['id'] = workbook.attrib['id']
                workbook_metadata['name'] = workbook.attrib['name']
                workbook_metadata['createdAt'] = workbook.attrib['createdAt']
                workbook_metadata['updatedAt'] = workbook.attrib['updatedAt']

                owner = workbook.find(".//{http://tableau.com/api}owner")
                if owner is not None:
                    workbook_metadata['ownerName'] = owner.attrib['name']

                datasources = workbook.findall(".//{http://tableau.com/api}datasource")
                datasource_types = []
                for datasource in datasources:
                    datasource_types.append(datasource.attrib.get('type', 'Unknown'))
                workbook_metadata['datasourceTypes'] = datasource_types

            print(workbook_metadata)
        except ET.ParseError as e:
            print("Error parsing XML response:", e)
            print("Response content:", response.content)
    else:
        print("Failed to get workbook metadata. Status code:", response.status_code)
        print("Response content:", response.content)


Response content: b'<?xml version=\'1.0\' encoding=\'UTF-8\'?><tsResponse xmlns="http://tableau.com/api" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://tableau.com/api https://help.tableau.com/samples/en-us/rest_api/ts-api_3_10.xsd"><workbook id="e3f507fd-12b9-4ab4-8d59-a8c1a613d58f" name="Account Engagement" contentUrl="AccountEngagement" webpageUrl="https://prod-apnortheast-a.online.tableau.com/#/site/jabocih360ecd23eb245/workbooks/1948144" showTabs="true" size="1" createdAt="2024-07-03T07:28:18Z" updatedAt="2024-07-03T07:28:18Z" encryptExtracts="false" defaultViewId="1a819f85-ee3b-486b-a541-23e8f09afd1b"><project id="3270c224-c139-4bca-920b-7818fc3fe2b8" name="default"/><owner id="f7101066-18a5-4f74-9d40-756a8fe1cbe5" name="jabocih360@luravel.com"/><tags/><views><view id="1a819f85-ee3b-486b-a541-23e8f09afd1b" name="Account Engagement" contentUrl="AccountEngagement/sheets/AccountEngagement" createdAt="2024-07-03T07:28:18Z" updatedAt="2024-07-03T07:28